In [1]:
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer
import pandas as pd
from tqdm.auto import tqdm
import jsonlines

In [10]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("pythainlp/wangchanglm-7.5B-sft-enth")
model = AutoModelForCausalLM.from_pretrained("pythainlp/wangchanglm-7.5B-sft-enth")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## Klon

In [77]:
ds = []
with jsonlines.open('../data_large/klong/klonnotag.json') as reader:
    for d in tqdm(reader):
       ds.append(d) 

0it [00:00, ?it/s]

In [78]:
ds[0]

[{'text': '<human>: แต่งกลอนแปดจากวรรคนี้ <context>:พอออกเรือเมื่อตวันสายัณห์ย่ำ <bot>:ลอองน้ำค้างย้อยเปนฝอยฝน ตลึงเหลียวเปลี่ยวเปล่าเมื่อคราวจน ไม่มีคนเกื้อหนุนกรุณา',
  'source': 'นิราศเจ้าฟ้า'},
 {'text': '<human>: แต่งกลอนแปดจากวรรคนี้ <context>:โอ้ธานีศรีอยุธมนุษย์แน่น <bot>:นับโกฏิแสนสาวแก่แซ่ภาษา จะหารักสักคนพอปนยา ไม่เห็นหน้านึกสอื้นฝืนฤทัย',
  'source': 'นิราศเจ้าฟ้า'},
 {'text': '<human>: แต่งกลอนแปดจากวรรคนี้ <context>:เสียแรงมีพี่ป้าหม่อมน้าสาว <bot>:ล้วนขาวขาวคำหวานน้ำตาลใส มายามยืดจืดเปรี้ยวไปเจียวใจ เหลืออาไลยลมปากจะจากจร',
  'source': 'นิราศเจ้าฟ้า'},
 {'text': '<human>: แต่งกลอนแปดจากวรรคนี้ <context>:ถึงวัดระฆังบังคมบรมธาตุ <bot>:แทบพระบาทบุษบงสุ์องค์อัปศร ไม่ทันลับกัปกัลป์พุทธันดร พระด่วนจรสู่สวรรค์ครรไล',
  'source': 'นิราศเจ้าฟ้า'},
 {'text': '<human>: แต่งกลอนแปดจากวรรคนี้ <context>:ละสมบัติขัติยาทั้งข้าบาท <bot>:โอ้อนาถนึกน่าน้ำตาไหล เปนสูญลับนับปีแต่นี้ไป เหลืออาไลยแล้วที่พระมีคุณ',
  'source': 'นิราศเจ้าฟ้า'},
 {'text': '<human>: แต่งกลอนแปดจากวรรคนี้ <context>

In [80]:
#test if masking correctly
l = tokenizer(ds[0][18090]['text'], add_special_tokens=False)['input_ids']
context_cue = tokenizer('<context>:')['input_ids'][1:]
human_cue = tokenizer('<human>:')['input_ids'][1:]
bot_cue = tokenizer('<bot>:')['input_ids'][1:]

def mask_labels(l, context_cue, human_cue, bot_cue):
    result = []
    i = 0
    while i < len(l):
        if (l[i:i+len(human_cue)] == human_cue)|((l[i:i+len(context_cue)] == context_cue)):
            while l[i:i+len(bot_cue)] != bot_cue:
                result.append(-100)
                i += 1
        else:
            result.append(l[i])
            i += 1
    return result

l2 = mask_labels(l, context_cue, human_cue, bot_cue)
l2

[-100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 1131,
 9932,
 1128,
 13,
 3748,
 16357,
 28945,
 35756,
 35312,
 14289,
 13334,
 1485,
 44461,
 6325,
 5834,
 83522,
 5014,
 10580,
 152460,
 17246,
 7210,
 2357,
 34597,
 23142,
 37405,
 6,
 80948,
 252126,
 83522,
 39061,
 6721,
 12079,
 7033,
 2165,
 6612]

In [81]:
' '.join([tokenizer.decode(i) for i in l2 if i!=-100])

'< bot > : จะ ชอบ ผิด ปรับ ไหม ไ ฉ น หน อ อ ้าย คน โ ง่ เอง อย่าง ด กด ต้น คอ  ถอ งอ ้าย พ่อ ตา เล่น ถึง เป็น ความ'

In [83]:
ds[0][18090]['text']

'<human>: แต่งกลอนแปดจากวรรคนี้ <context>:เฮ้ยอ้ายเจ้ามังคลาสานุศิษย์ <bot>:จะชอบผิดปรับไหมไฉนหนอ อ้ายคนโง่เองอย่างดกดต้นคอ ถองอ้ายพ่อตาเล่นถึงเป็นความ'

In [84]:
dset = Dataset.from_list(ds[0])
dset

Dataset({
    features: ['text', 'source'],
    num_rows: 56441
})

In [88]:
dset[18090]

{'text': '<human>: แต่งกลอนแปดจากวรรคนี้ <context>:เฮ้ยอ้ายเจ้ามังคลาสานุศิษย์ <bot>:จะชอบผิดปรับไหมไฉนหนอ อ้ายคนโง่เองอย่างดกดต้นคอ ถองอ้ายพ่อตาเล่นถึงเป็นความ',
 'source': 'พระอภัยมณี'}

In [92]:
def preprocess_sft(example):
    #coutn token
    example['nb_tokens'] = len(tokenizer(example['text'])['input_ids'])
    return example

def is_sublist(sublist, mainlist):
  if len(sublist) > len(mainlist):
    return False
  for i in range(len(mainlist) - len(sublist) + 1):
    if sublist == mainlist[i:i + len(sublist)]:
      return True
  return False

def filter_malformed_bot(example):
    #filter out those with malformed <bot>:
    bot_tokens = tokenizer('<bot>:', add_special_tokens=False)['input_ids']
    text_tokens = tokenizer(example['text'], add_special_tokens=False)['input_ids']
    if is_sublist(bot_tokens,text_tokens):
        return True
    else:
        return False

dset_klon_before = dset.map(preprocess_sft)
dset_klon = dset_klon_before.filter(filter_malformed_bot)

  0%|          | 0/56441 [00:00<?, ?ex/s]

  0%|          | 0/57 [00:00<?, ?ba/s]

In [93]:
dset_klon_before, dset_klon

(Dataset({
     features: ['text', 'source', 'nb_tokens'],
     num_rows: 56441
 }),
 Dataset({
     features: ['text', 'source', 'nb_tokens'],
     num_rows: 56440
 }))

In [94]:
dset_klon.shuffle()[0]

{'text': '<human>: แต่งกลอนแปดจากวรรคนี้ <context>:บาทหลวงเดินขัดกระบี่ที่แม่ทัพ <bot>:ตรวจกำกับพลไพร่ไล่พหล พวกกองหน้าแต่บรรดาหุ่นพยนต์ ทั้งคงทนแทงฟันไม่บรรลัย',
 'source': 'พระอภัยมณี',
 'nb_tokens': 61}

In [95]:
pd.Series(dset_klon['nb_tokens']).quantile([i/1000 for i in range(1000)]),\
pd.Series(dset_klon['nb_tokens']).max()

(0.000    46.0
 0.001    50.0
 0.002    52.0
 0.003    52.0
 0.004    53.0
          ... 
 0.995    76.0
 0.996    77.0
 0.997    77.0
 0.998    78.0
 0.999    79.0
 Length: 1000, dtype: float64,
 85)

In [96]:
#100% are less than 512 tokens
(pd.Series(dset_klon['nb_tokens'])<512).mean()

1.0

## Klong4

In [97]:
ds = []
with jsonlines.open('../data_large/klong/klong4notag.json') as reader:
    for d in tqdm(reader):
       ds.append(d) 

0it [00:00, ?it/s]

In [98]:
#test if masking correctly
l = tokenizer(ds[0][100]['text'], add_special_tokens=False)['input_ids']
context_cue = tokenizer('<context>:')['input_ids'][1:]
human_cue = tokenizer('<human>:')['input_ids'][1:]
bot_cue = tokenizer('<bot>:')['input_ids'][1:]

def mask_labels(l, context_cue, human_cue, bot_cue):
    result = []
    i = 0
    while i < len(l):
        if (l[i:i+len(human_cue)] == human_cue)|((l[i:i+len(context_cue)] == context_cue)):
            while l[i:i+len(bot_cue)] != bot_cue:
                result.append(-100)
                i += 1
        else:
            result.append(l[i])
            i += 1
    return result

l2 = mask_labels(l, context_cue, human_cue, bot_cue)
l2

[-100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 1131,
 9932,
 1128,
 13,
 3561,
 61941,
 6964,
 75713,
 5109,
 58117,
 13778,
 11415,
 37917,
 43832,
 2357,
 57294,
 25289,
 3561,
 133082,
 103792,
 138814,
 3100,
 23273,
 86625,
 1731,
 19849,
 75168,
 15414,
 92410,
 49784,
 67736,
 12101,
 27520,
 49408,
 13778,
 11415,
 13420,
 2357,
 867,
 206677,
 29507,
 159129,
 2598,
 6,
 84077,
 3987,
 33032,
 15163,
 15163,
 5109]

In [99]:
' '.join([tokenizer.decode(i) for i in l2 if i!=-100])

'< bot > : ช ฎ า ธา ร ท่าน แ ฮ ทั้ง แป ด สํา มั ช ฌ าญ เส พ สร้าง อั ม พร วิ่ง วง ทะ ยาน ยัง เม ฆ ไป แ ฮ ป ด ที่ ออน อัน ร้า ง  รีบ ห ื้อ สม สม ร'

In [100]:
ds[0][100]['text']

'<human>: แต่งโคลงสี่สุภาพจากเริ่มวรรคนี้ <context>:ฤาษีสรัพศาสตร์ไท้ <bot>:ชฎาธาร ท่านแฮ ทั้งแปดสำมัชฌาญ เสพสร้าง อัมพรวิ่งวงทะยาน ยังเมฆ ไปแฮ ปดที่ออนอันร้าง รีบหื้อสมสมร'

In [101]:
dset = Dataset.from_list(ds[0])
dset

Dataset({
    features: ['text', 'source'],
    num_rows: 1326
})

In [102]:
dset[0]

{'text': '<human>: แต่งโคลงสี่สุภาพจากเริ่มวรรคนี้ <context>:กชกรต่างแต่ตั้ง <bot>:ศิรษา นบพุทธธรรมสาวกา แผ่นเผ้า สนำสลูเบิกนามมา ขอมเรียก รักแฮ ไทด่ำบลเมิงเป้า ผ่านไว้วิวรณ์',
 'source': 'โคลงนิราศหริภุญชัย'}

In [103]:
def preprocess_sft(example):
    #coutn token
    example['nb_tokens'] = len(tokenizer(example['text'])['input_ids'])
    return example

def is_sublist(sublist, mainlist):
  if len(sublist) > len(mainlist):
    return False
  for i in range(len(mainlist) - len(sublist) + 1):
    if sublist == mainlist[i:i + len(sublist)]:
      return True
  return False

def filter_malformed_bot(example):
    #filter out those with malformed <bot>:
    bot_tokens = tokenizer('<bot>:', add_special_tokens=False)['input_ids']
    text_tokens = tokenizer(example['text'], add_special_tokens=False)['input_ids']
    if is_sublist(bot_tokens,text_tokens):
        return True
    else:
        return False

dset_klong4_before = dset.map(preprocess_sft)
dset_klong4 = dset_klong4_before.filter(filter_malformed_bot)

  0%|          | 0/1326 [00:00<?, ?ex/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [107]:
dset_klong4_before, dset_klong4

(Dataset({
     features: ['text', 'source', 'nb_tokens'],
     num_rows: 1326
 }),
 Dataset({
     features: ['text', 'source', 'nb_tokens'],
     num_rows: 1326
 }))

In [104]:
#100% is less than 512
pd.Series(dset_klong4['nb_tokens']).quantile([i/10 for i in range(10)]),\
pd.Series(dset_klong4['nb_tokens']).max()

(0.0    60.0
 0.1    69.0
 0.2    71.0
 0.3    73.0
 0.4    75.0
 0.5    76.0
 0.6    77.0
 0.7    79.0
 0.8    81.0
 0.9    83.0
 dtype: float64,
 100)

## Combine Them

In [105]:
dset_klong4

Dataset({
    features: ['text', 'source', 'nb_tokens'],
    num_rows: 1326
})

In [106]:
dset_klon

Dataset({
    features: ['text', 'source', 'nb_tokens'],
    num_rows: 56440
})

In [109]:
from datasets import concatenate_datasets
dset_klongklon = concatenate_datasets([dset_klon, dset_klong4])
dset_klongklon = dset_klongklon.train_test_split(test_size=166, seed=125)
dset_klongklon

DatasetDict({
    train: Dataset({
        features: ['text', 'source', 'nb_tokens'],
        num_rows: 57600
    })
    test: Dataset({
        features: ['text', 'source', 'nb_tokens'],
        num_rows: 166
    })
})

In [111]:
dset_klongklon.push_to_hub('pythainlp/klongklon', private=True)

Pushing split train to the Hub.


Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Deleting unused files from dataset repository:   0%|          | 0/1 [00:00<?, ?it/s]

Pushing split test to the Hub.


Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Deleting unused files from dataset repository:   0%|          | 0/1 [00:00<?, ?it/s]

## Test Tokenization Works

In [113]:
import copy
ds = load_dataset('pythainlp/klongklon')
cpu_cores = 8
# Preprocess the dataset.
def mask_labels(l, context_cue, human_cue, bot_cue):
#     print(l)
    result = []
    i = 0
    while i < len(l):
        if (l[i:i+len(human_cue)] == human_cue)|((l[i:i+len(context_cue)] == context_cue)):
            while l[i:i+len(bot_cue)] != bot_cue:
                result.append(-100)
                i += 1
        else:
            result.append(l[i])
            i += 1
    return result
        
def preprocess_function(example):
    tokenized_qa = tokenizer(example['text']+tokenizer.eos_token, 
                            truncation=True, 
                            padding="max_length",
                            max_length=512,
                            add_special_tokens=False
                            )
    labels = copy.deepcopy(tokenized_qa['input_ids'])
    labels = mask_labels(labels, 
              tokenizer('<context>:', add_special_tokens=False)['input_ids'],
              tokenizer('<human>:', add_special_tokens=False)['input_ids'],
              tokenizer('<bot>:', add_special_tokens=False)['input_ids']
             )
    labels = [-100 if i==tokenizer.pad_token_id else i for i in labels]
    return {
        "input_ids": tokenized_qa["input_ids"],
        "attention_mask": tokenized_qa["attention_mask"],
        "labels": labels,
    }

tokenized_ds = ds.map(preprocess_function, 
                      batched=False, 
                      num_proc=1, 
                      remove_columns=ds['train'].column_names)

Using custom data configuration pythainlp--klongklon-54f5cafac27cbdfb
Found cached dataset parquet (/home/charipol/.cache/huggingface/datasets/pythainlp___parquet/pythainlp--klongklon-54f5cafac27cbdfb/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/166 [00:00<?, ?ex/s]

  0%|          | 0/57600 [00:00<?, ?ex/s]

In [114]:
tokenized_ds

DatasetDict({
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 166
    })
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 57600
    })
})